# Libraries used
PV --> Running 3.10.10
Running Kernel3.9.13 base anaconda

#perform al pip installs in one go comment out if already installed
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install sklearn
!pip install scipy
!pip install statsmodels
!pip install plotly
!pip install cufflinks
!pip install squarify
!pip install yellowbrick
!pip install lazypredict
!pip install pandas_profiling

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pyplot as plt
import plotly.io as pio
#import squarify #treemap
import os
import matplotlib
import warnings



#to enable the inline plotting
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

sns.set_style("darkgrid")

In [2]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import classification_report

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures


from scipy.stats import normaltest

from pandas_profiling import ProfileReport

from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.style.palettes import PALETTES, SEQUENCES, color_palette

import lazypredict
from lazypredict.Supervised import LazyClassifier

warnings.simplefilter(action='ignore', category=FutureWarning)

# Functions


In [3]:
# Function for EDA. Using the display() function to have  well-formatted tables. We are mainly using pandas to explore the datasets

def dataset_description(df_target):

    print('This is the Dataset shape: %s\n' % (df_target.shape, ))
    print('Dataset columns: %s\n' % df_target.columns)

    print('\nColumns description:\n')
    display(df_target.info())
    display(df_target.describe())  # describe the dataset

    print('\nNull values:\n')
    display(df_target.isnull().sum())  # Identify null values

#function performing a quick check on df_inspection to have best of pandas functions separated by a line
def quick_check(dataframe):
    print('First 5 rows %s\n')
    print(dataframe.head(2))
    print("=====================================")
    print('Dataframe shape %s\n')
    print(dataframe.shape)
    print("=====================================")
    print('Dataframe describe categorical %s\n')
    print(dataframe.describe(include=['O']))
    print("=====================================")
    print('Dataframe null values %s\n')
    print(dataframe.isnull().sum())
    print("=====================================")
    print('Dataframe value counts %s\n')
    print(dataframe.value_counts())
    print("=====================================")

#stats function
def stats(dataframe):
    print('Dataframe correlation %s\n')
    print(dataframe.corr())
    print("=====================================")
    print('Dataframe covariance %s\n')
    print(dataframe.cov())
    print("=====================================")
    print('Dataframe skew %s\n')
    print(dataframe.skew())
    print("=====================================")
    print('Dataframe kurtosis %s\n')
    print(dataframe.kurt())
    print("=====================================")

#create a function to normalize characters from a dataset's column in Spanish
def normalize_characters(df, column):
    df[column] = df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace('á', 'a')
    df[column] = df[column].str.replace('é', 'e')
    df[column] = df[column].str.replace('í', 'i')
    df[column] = df[column].str.replace('ó', 'o')
    df[column] = df[column].str.replace('ú', 'u')
    df[column] = df[column].str.replace('ñ', 'n')
    df[column] = df[column].str.replace('ü', 'u')
    df[column] = df[column].str.replace('ç', 'c')
    df[column] = df[column].str.replace('(', '')
    df[column] = df[column].str.replace(')', '')
    df[column] = df[column].str.replace('\'', '')
    df[column] = df[column].str.replace('´', '')
    df[column] = df[column].str.replace('`', '')
    df[column] = df[column].str.replace('’', '')
    return df.head(2)

#create function to change detypes in64 to int32 in a df
def change_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

def outlier_function(df, col_name):
    """ this function detects first and third quartile and interquartile range for a given column of a dataframe
    then calculates upper and lower limits to determine outliers conservatively
    returns the number of lower and uper limit and number of outliers respectively"""
    first_quartile = np.percentile(np.array(df[col_name].tolist()), 25)
    third_quartile = np.percentile(np.array(df[col_name].tolist()), 75)
    IQR = third_quartile - first_quartile
                        
    upper_limit = third_quartile+(3*IQR)
    lower_limit = first_quartile-(3*IQR)
    outlier_count = 0
                    
    for value in df[col_name].tolist():
        if (value < lower_limit) | (value > upper_limit):
            outlier_count +=1
        else:
            pass
    return lower_limit, upper_limit, outlier_count

In [4]:
#show all print outputs when using a function
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#display all columns
pd.set_option('display.max_columns', None)


# 2. Data Collection and Understanding 

## Network file

The network.csv file contains a static picture of the gas pipeline network. Every row corresponds to a pipe and has a unique PipeId identifier. The table has 1.446.529 pipes. 

The columns describe relevant features of each pipe. The complete list is: 

PipeId - unique identifier for the pipe 

Province - Spanish province where the pipe is located 

Town - Town or city where the pipe is located 

YearBuilt - Year in which the pipe was built and installed 

Material - Material in which the pipe is built 

GasType - Type of gas that runs through the pipe 

Diameter - diameter of the pipe 

Length - Length of the pipe 

Pressure - Pressure of the gas that runs through the pipe (bar) 

NumConnections - Number of connections (external). 

NumConnectionsUnder - Number of connections (internal and buried) 

BoolBridle  - Whether the pipe is bridled (True) or welded (False) 

## Inspection file

MaintenanceId - unique identifier for the inspection operation 

InspectionYear - year in which the inspection took place 

InspectionDate - date in which the inspection took place 

MonthsLastRev - number of months elapsed since the last previous inspection. 

Severity - Severity of the damage found (1: most severe, 3: least severe) 

Incidence - Boolean whether an incident was found on the revision (1) or not (0). 

# Merging datasets

In [5]:
#point to the folder where the data is stored for Pedro
os.chdir(r"C:\Users\pedro\datathon")

# Loading inspection data
df_inspection = pd.read_csv('inspections.csv')

# Loading network data
df_network = pd.read_csv('network.csv')


In [ ]:
#point to the folder where the data is stored for Pedro
os.chdir(r"C:\Users\pedro\datathon\Datathon-Rules-and-Documentation") 

#Loading sample_submission
df_sample_submission = pd.read_csv('sample_submission.csv')

#point to the folder where the data is stored For Juan
os.chdir(r"C:\Users\JuanHorrillo\OneDrive - IE Students\Documents\Masters\Sustainability\Notebook")

# Loading inspection data
df_inspection = pd.read_csv('inspections.csv')

# Loading network data
df_network = pd.read_csv('network.csv')

In [ ]:
quick_check(df_inspection)


The function shows we have 6345344 rows for the inspections, among each 4179 appear to be unique and the most repeated one is ZRV-00002121 on 2014-05-05


In [ ]:
#searching for MaintenanceId ZRV-00002121 in df_inspection
df_inspection[df_inspection['MaintenanceId'] == 'ZRV-00002121']

In [ ]:
df_inspection[df_inspection['MaintenanceId'] == 'ZRV-00002121'].shape

In [ ]:
#plotting the distribution of MaintenanceId accross the years for ZRV-00002121 in df_inspection
df_inspection[df_inspection['MaintenanceId'] == 'ZRV-00002121'].groupby('InspectionYear')['MaintenanceId'].count().plot(kind='bar')

In [ ]:
#plotting the distribution of MaintenanceId accross the years
df_inspection.groupby('InspectionYear')['MaintenanceId'].count().plot(kind='bar')

Seems like not so many inspections were carried in 2010, we shall decide on whether to keep this dimension or not

In [ ]:
stats(df_inspection)

 A positive kurtosis in a dataset implies that the data is more heavily concentrated around the mean than a normal distribution. This means that there are more outliers in the dataset and that the tails of the distribution are longer and fatter than a normal distribution. Months last revision is to be examined on this basis

In [ ]:
#plot histogram with many bins of MonthsLastRev in  df_inspection
df_inspection['MonthsLastRev'].hist(bins=100)

The shape of the histogram reveals the most frequent inspections take place every 2 years or 24 months with some outliers hence affecting the normal distribution

In [ ]:
#plot violin plot of PipeId with Severity=1 in df_inspection
sns.violinplot(x='PipeId', y='MonthsLastRev', data=df_inspection[df_inspection['Severity'] == 1])

#to work out better in final EDA

In [ ]:
quick_check(df_network)

In the network file we find 1446539 rows corresponding to all pipes, this is the master file for pipes.
We find the most frequent Town is Madrid but the most frequent province is Barcelona. 
There are a total of 1972 Spanish towns in this dataset and 38 provinces
We are missing the Basque Country, Ceuta, Melilla, the Canary Islands and Balearic Islands.



In [ ]:
#plot a bar plot with the number of pipes per Town focusing in Madrid and Barcelona  in df_network
df_network.groupby('Town')['PipeId'].count().sort_values(ascending=False).head(10).plot(kind='bar')

In [ ]:
#plot a pie chart with the number of pipes per Province in df_network 
df_network.groupby('Province')['PipeId'].count().sort_values(ascending=False).plot(kind='pie')


In [ ]:
stats(df_network)

In [ ]:
#join Inspection and network datasets on PipeId to create our TRAIN dataset
train = pd.merge(df_inspection, df_network, on='PipeId', how='right')

In [ ]:
train.shape

In [ ]:
df_inspection.shape

JF: There are 18K null values that do not have information in the df_network so our TRAIN data set is now bigger and we will need to deal with the nulls further down.

In [ ]:
#count null values in train dataset
train.isnull().sum()


In [ ]:
#point to the folder where the data is stored for Pedro
os.chdir(r"C:\Users\pedro\datathon\Datathon-Rules-and-Documentation")

# Loading inspection data
df_sample = pd.read_csv('sample_submission.csv')

In [ ]:
#point to the folder where the data is stored for Juan
#os.chdir(r"C:\Users\JuanHorrillo\OneDrive - IE Students\Documents\Masters\Sustainability\Notebook")

# Loading inspection data
#df_sample = pd.read_csv('sample_submission.csv')

In [ ]:
df_sample.shape

In [ ]:
df_sample.head()

In [ ]:
#count the Incidence values in test dataset
df_sample['Incidence'].value_counts()



Test data set is also unbalanced 

Creating the TEST df which will be used to test our model.  Thisis made of the sample submission with the columns from the network

In [ ]:
#create a dataframe with the unique PipeId from df_sample and a dataframe with the PipeId from df_network 
sample_unique = df_sample['PipeId'].unique()
network_unique = df_network['PipeId'].unique()
#check if all the values in sample_unique are in network_unique
np.all(np.isin(sample_unique, network_unique))

JF: All the PipeId on the df_sample have a PipeId on the network

In [ ]:
#join both datasets on PipeId
test = pd.merge(df_network , df_sample, on='PipeId', how='right')

In [ ]:
test.shape

In [ ]:
test.head()

In [ ]:
df_sample.head()

In [ ]:
#Check for null values in test dataset
test_null = test.isnull().sum()    
test_null.head()

JF: No null Values in TEST data set

In [ ]:
#creating a subset for altering the dataset after initial EDA
train_copy = train.copy()

In [ ]:
#find the columns with null values in train_copy
train_copy.isnull().sum()

In [ ]:
#show me a head of train_copy with YearBuilt == 2020
train_copy[train_copy['YearBuilt'] > 2020].head()

In [ ]:
#count the PipeID that have YearBuilt equal to 2021 and InspectionYear equal to NaN
train_copy[(train_copy['YearBuilt'] > 2020) & (train_copy['InspectionYear'].isnull())].shape


JF: There are 2026 pipes that were built after 2020 in the train data set, these came from the network data set and will be considered new pipes that do not have inspection data yet. We will assume that the inspection year for those was the year of installation (you inspect them when you install them)

In [ ]:
#show me nulls for InspectionYear for every year in train_copy
train_copy.groupby('InspectionYear')['InspectionYear'].count()

In [ ]:
#do a range in the train_copy for the YearBuilt column if the value is greater than 2020 and the InspectionYear is equal to Nan, change the value of InspectionYear to YearBuilt

#for i in train_copy['YearBuilt']:
#    if i > 2020:
#        train_copy.loc[(train_copy['YearBuilt'] > 2020) & (train_copy['InspectionYear'].isnull()), 'InspectionYear'] = i

In [ ]:
#show me the YearBuilt values for the null vales in InspectionYear
train_copy[train_copy['InspectionYear'].isnull()]['YearBuilt'].value_counts()


In [ ]:
#remove from train_copy the rows where InspectionYear is equal to NaN and yearBuilt is not null
train_copy = train_copy[~((train_copy['InspectionYear'].isnull()) & (train_copy['YearBuilt'].notnull()))]

We do not have info for these pipes so I will remove them all as it is impossible to find out the year built

In [ ]:
train_copy.isnull().sum()

In [ ]:
#count the PipeId in inspection dataset that YearBuilt is equal to 2020 and inspection year is equal to 2021
train_copy[(train_copy['YearBuilt'] > 2020) & (train_copy['InspectionYear'] == 2021)].shape



#perform al pip installs in one go comment out if already installed
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install sklearn
!pip install scipy
!pip install statsmodels
!pip install plotly
!pip install cufflinks
!pip install squarify
!pip install yellowbrick
!pip install lazypredict
!pip install pandas_profiling

In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import classification_report

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import StandardScaler


from scipy.stats import normaltest

from pandas_profiling import ProfileReport

from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.style.palettes import PALETTES, SEQUENCES, color_palette

import lazypredict
from lazypredict.Supervised import LazyClassifier

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Function for EDA. Using the display() function to have  well-formatted tables. We are mainly using pandas to explore the datasets

def dataset_description(df_target):

    print('This is the Dataset shape: %s\n' % (df_target.shape, ))
    print('Dataset columns: %s\n' % df_target.columns)

    print('\nColumns description:\n')
    display(df_target.info())
    display(df_target.describe())  # describe the dataset

    print('\nNull values:\n')
    display(df_target.isnull().sum())  # Identify null values

#function performing a quick check on df_inspection to have best of pandas functions separated by a line
def quick_check(dataframe):
    print('First 5 rows %s\n')
    print(dataframe.head(2))
    print("=====================================")
    print('Dataframe shape %s\n')
    print(dataframe.shape)
    print("=====================================")
    print('Dataframe describe categorical %s\n')
    print(dataframe.describe(include=['O']))
    print("=====================================")
    print('Dataframe null values %s\n')
    print(dataframe.isnull().sum())
    print("=====================================")
    print('Dataframe value counts %s\n')
    print(dataframe.value_counts())
    print("=====================================")

#stats function
def stats(dataframe):
    print('Dataframe correlation %s\n')
    print(dataframe.corr())
    print("=====================================")
    print('Dataframe covariance %s\n')
    print(dataframe.cov())
    print("=====================================")
    print('Dataframe skew %s\n')
    print(dataframe.skew())
    print("=====================================")
    print('Dataframe kurtosis %s\n')
    print(dataframe.kurt())
    print("=====================================")

#create a function to normalize characters from a dataset's column in Spanish
def normalize_characters(df, column):
    df[column] = df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace('á', 'a')
    df[column] = df[column].str.replace('é', 'e')
    df[column] = df[column].str.replace('í', 'i')
    df[column] = df[column].str.replace('ó', 'o')
    df[column] = df[column].str.replace('ú', 'u')
    df[column] = df[column].str.replace('ñ', 'n')
    df[column] = df[column].str.replace('ü', 'u')
    df[column] = df[column].str.replace('ç', 'c')
    df[column] = df[column].str.replace('(', '')
    df[column] = df[column].str.replace(')', '')
    df[column] = df[column].str.replace('\'', '')
    df[column] = df[column].str.replace('´', '')
    df[column] = df[column].str.replace('`', '')
    df[column] = df[column].str.replace('’', '')
    return df

#create function to change detypes in64 to int32 in a df
def change_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

# 2. Data Collection and Understanding 

The network.csv file contains a static picture of the gas pipeline network. Every row corresponds to a pipe and has a unique PipeId identifier. The table has 1.446.529 pipes. 

The columns describe relevant features of each pipe. The complete list is: 

## Inspection file

# Merging datasets

In [ ]:
#point to the folder where the data is stored for Pedro
os.chdir(r"C:\Users\pedro\datathon\Datathon-Rules-and-Documentation") 

#Loading sample_submission
df_sample_submission = pd.read_csv('sample_submission.csv')

In [ ]:
quick_check(df_inspection)


In [ ]:
#searching for MaintenanceId ZRV-00002121 in df_inspection
df_inspection[df_inspection['MaintenanceId'] == 'ZRV-00002121']

In [ ]:
#plotting the distribution of MaintenanceId accross the years for ZRV-00002121 in df_inspection
df_inspection[df_inspection['MaintenanceId'] == 'ZRV-00002121'].groupby('InspectionYear')['MaintenanceId'].count().plot(kind='bar')

Seems like not so many inspections were carried in 2010, we shall decide on whether to keep this dimension or not

In [ ]:
stats(df_inspection)

In [ ]:
#plot histogram with many bins of MonthsLastRev in  df_inspection
df_inspection['MonthsLastRev'].hist(bins=100)

In [ ]:
#JUAN we kill more than 40 months revision

#to work out better in final EDA

In the network file we find 1446539 rows corresponding to all pipes, this is the master file for pipes.
We find the most frequent Town is Madrid but the most frequent province is Barcelona. 
There are a total of 1972 Spanish towns in this dataset and 38 provinces
We are missing the Basque Country, Ceuta, Melilla, the Canary Islands and Balearic Islands.



In [ ]:
#plot a bar plot with the number of pipes per Town focusing in Madrid and Barcelona  in df_network
df_network.groupby('Town')['PipeId'].count().sort_values(ascending=False).head(10).plot(kind='bar')

In [ ]:
stats(df_network)

In [ ]:
train.shape 

JF: There are 18K null values that do not have information in the df_network so our TRAIN data set is now bigger and we will need to deal with the nulls further down.

In [ ]:
#point to the folder where the data is stored for Pedro
os.chdir(r"C:\Users\pedro\datathon\Datathon-Rules-and-Documentation")

# Loading inspection data
df_sample = pd.read_csv('sample_submission.csv')

In [ ]:
df_sample.shape

In [ ]:
#count the Incidence values in test dataset
df_sample['Incidence'].value_counts()



Creating the TEST df which will be used to test our model.  Thisis made of the sample submission with the columns from the network

JF: All the PipeId on the df_sample have a PipeId on the network

In [ ]:
#join both datasets on PipeId
test = pd.merge(df_network , df_sample, on='PipeId', how='right')

In [ ]:
test.head()

In [ ]:
#Check for null values in test dataset
test_null = test.isnull().sum()    
test_null.head()

In [ ]:
#creating a subset for altering the dataset after initial EDA
train_copy = train.copy()

In [ ]:
#show me a head of train_copy with YearBuilt == 2020
train_copy[train_copy['YearBuilt'] > 2020].head()

JF: There are 2026 pipes that were built after 2020 in the train data set, these came from the network data set and will be considered new pipes that do not have inspection data yet. We will assume that the inspection year for those was the year of installation (you inspect them when you install them)

In [ ]:
#show me nulls for InspectionYear for every year in train_copy
train_copy.groupby('InspectionYear')['InspectionYear'].count()

In [ ]:
train_copy.isnull().sum()

In [ ]:
#show me a head of train_copy with YearBuilt higher than InspectionYear and count them
train_copy[(train_copy['YearBuilt'] > train_copy['InspectionYear'])].shape

# Feature Engineering on combined dataset

As a result of merging both datasets we now have pipeline duplicates per each maintenace_id operation. Before getting rid of the duplicates, we want to engineer some metrics interesting to the model such as number of operations, number of incidents and average risk based on severity*incidence

In [ ]:
#create a new column counting the number of inspections (MaintenanceId) per PipeId
train_copy['No_Inspections'] = train_copy.groupby('PipeId')['MaintenanceId'].transform('count')
#aggregate the number of Incidents per pipe in a new column and place it in the fourth position
train_copy['No_Incidents'] = train_copy.groupby('PipeId')['Incidence'].transform('sum')
#place the new columns in the third position
cols = list(train_copy.columns.values)
cols.pop(cols.index('No_Incidents'))
train_copy = train_copy[['PipeId', 'MaintenanceId', 'No_Inspections', 'No_Incidents', 'InspectionYear', 'InspectionDate',
       'MonthsLastRev', 'Severity', 'Incidence', 'Province', 'Town',
       'YearBuilt', 'Material', 'GasType', 'Diameter', 'Length', 'Pressure',
       'NumConnections', 'NumConnectionsUnder', 'BoolBridle']]
#show head of rows only where No_Incidents is greater than 0
train_copy[train_copy['No_Incidents'] > 2].head(2)


In [ ]:
train_copy.columns

In [ ]:
#create a column named average_severity_pipe that calculates the average severity per pipe aggregating all severities
train_copy['average_severity_pipe'] = train_copy.groupby('PipeId')['Severity'].transform('mean')
#show head of rows only where mean has a decimal value
train_copy[train_copy['average_severity_pipe'] % 1 != 0].head(10)
#place the new column in 7th position
cols = list(train_copy.columns.values)
cols.pop(cols.index('average_severity_pipe'))
train_copy = train_copy[['PipeId', 'No_Inspections', 'No_Incidents', 'InspectionYear',
       'InspectionDate', 'MonthsLastRev', 'average_severity_pipe','Severity', 'Incidence', 'Province',
       'Town', 'YearBuilt', 'Material', 'GasType', 'Diameter', 'Length',
       'Pressure', 'NumConnections', 'NumConnectionsUnder', 'BoolBridle']]


In [ ]:
#create a column taking average severity and number of total incidences per pipe multiplying them and naming it as risk_(s*i)
train_copy['relative_risk'] = train_copy['average_severity_pipe'] * train_copy['No_Incidents'] 
#position the new column in the 7th position
cols = list(train_copy.columns.values)
cols.pop(cols.index('relative_risk'))
train_copy = train_copy[['PipeId', 'No_Inspections', 'No_Incidents', 'InspectionYear',
       'InspectionDate', 'MonthsLastRev','relative_risk', 'average_severity_pipe', 'Severity',
       'Incidence', 'Province', 'Town', 'YearBuilt', 'Material', 'GasType',
       'Diameter', 'Length', 'Pressure', 'NumConnections',
       'NumConnectionsUnder', 'BoolBridle']]

This new column allows us to have a measure of risk by adding the mean aggregate of severity for an specific pipe and the aggregate number of incidents that we just created above

In [ ]:
#show head of rows only where Risk_S*I is greater than 0
train_copy[train_copy['relative_risk'] > 12].head(2)

In [ ]:
import plotly.express as px
#show a treemap when relative_risk is greater than 12 showing the Material and the age of construction
fig = px.treemap(train_copy[train_copy['relative_risk'] > 9], path=['Material', 'YearBuilt'], values='relative_risk')
fig.show()



PE and FD tendo to fail more when getting closer to 30 years old

In [ ]:
#show a treemap when relative_risk is greater than 12 showing the Provinces and Towns
fig = px.treemap(train_copy[train_copy['relative_risk'] > 9], path=['Province', 'Town'], values='relative_risk')
fig.show()


Based on risk, Madrid, Barcelona, Valencia, Girona and Sevilla should be studied more in detail when elaborating a maintenance plan

In [ ]:
#what is the last inspection year for material FD
train_copy[train_copy['Material'] == 'FD']['InspectionYear'].min()

We can see that  for a high risk factor, the most common materials are PE and FD with a mean of year built in the 90s. The pipes do not seem to be abandoned as the last year of inspection seems to be 2010

In [ ]:
train_copy.columns

In [ ]:
#create a new column taking Risk_S*I and dividing it by No_Inspections naming it as Risk_S*I/Inspections and placing it in 6th position
train_copy['preventive_maintenance_rate'] = train_copy['relative_risk'] / train_copy['No_Inspections']
cols = list(train_copy.columns.values)
cols.pop(cols.index('preventive_maintenance_rate'))
train_copy = train_copy[['PipeId', 'No_Inspections', 'No_Incidents', 'InspectionYear', 'preventive_maintenance_rate',
       'InspectionDate', 'MonthsLastRev', 'relative_risk',
       'average_severity_pipe', 'Severity', 'Incidence', 'Province', 'Town',
       'YearBuilt', 'Material', 'GasType', 'Diameter', 'Length', 'Pressure',
       'NumConnections', 'NumConnectionsUnder', 'BoolBridle']]

#show head of rows only where Risk_S*I/Inspections is greater than 0
train_copy[train_copy['preventive_maintenance_rate'] > 0].head(5)

In this new column we can see a dimension of how good it has been previous preventive maintenance  by dividing the relative risk factor (mean average risk per pipe*No_incidents) / No_inspections.
The higher this index, the less maintenance it has undergone, hence the risk is not being mitigated on an active basis
This graph can help us allocate better the maintenance in our business case using the Paretto rule 


In [ ]:
#graph a treemap showing the Province and Towns where preventive_maintenance_rate is higher than 2
fig = px.treemap(train_copy[train_copy['preventive_maintenance_rate'] > 2], path=['Province', 'Town'], values='preventive_maintenance_rate')
fig.show()



In [ ]:
train_copy.columns

In [ ]:
#creating a new column named probability with No_Incidents divided by Inspections
train_copy['Probability_rate'] = train_copy['No_Incidents'] / train_copy['No_Inspections']
#place column in 3rd position
cols = list(train_copy.columns.values)
cols.pop(cols.index('Probability_rate'))
train_copy = train_copy[['PipeId', 'No_Inspections', 'No_Incidents', 'InspectionYear', 'Probability_rate',
       'preventive_maintenance_rate', 'InspectionDate', 'MonthsLastRev',
       'relative_risk', 'average_severity_pipe', 'Severity', 'Incidence',
       'Province', 'Town', 'YearBuilt', 'Material', 'GasType', 'Diameter',
       'Length', 'Pressure', 'NumConnections', 'NumConnectionsUnder',
       'BoolBridle']]
#show head of rows only where Probability is greater than 0
train_copy[train_copy['Probability_rate'] > 0].head(5)

In [ ]:
#graph a treemap showing the Province and Towns where Probability is higher than 4
fig = px.treemap(train_copy[train_copy['Probability_rate'] > .5], path=['Province', 'Town'], values='Probability_rate')
fig.show()

Having created risk and probability dimensions, we can compute a matrix to be assigned to the provinces for prioritization
The matrix should include high risk, high probability to low risk and low probability    

In [ ]:
#create a matrix with columns relative_risk and probability_rate for each province
train_copy.groupby('Province')[['relative_risk', 'Probability_rate']].mean()


In [ ]:
#create a matrix with columns relative_risk and probability_rate for each pipe material
train_copy.groupby('Material')[['relative_risk', 'Probability_rate']].mean()

In [ ]:
train_copy.columns  

In [ ]:
#creating a column with the Average of MonthsLastRev grouping per PipeId 
train_copy['Average_MonthsLastRev'] = train_copy.groupby('PipeId')['MonthsLastRev'].transform('mean')
cols = list(train_copy.columns.values)
cols.pop(cols.index('Average_MonthsLastRev'))
train_copy = train_copy[['PipeId', 'No_Inspections', 'No_Incidents', 'InspectionYear',
       'Probability_rate', 'preventive_maintenance_rate', 'InspectionDate', 'Average_MonthsLastRev',
       'MonthsLastRev', 'relative_risk', 'average_severity_pipe', 'Severity',
       'Incidence', 'Province', 'Town', 'YearBuilt', 'Material', 'GasType',
       'Diameter', 'Length', 'Pressure', 'NumConnections',
       'NumConnectionsUnder', 'BoolBridle']]
train_copy.head(1)

With this new column we know if the pipe is being inspected frewuently or not taking as a thresold 24 months. We create a ne column that takes this threshold into consideration and creates a boolean


In [ ]:
#create a new column named pipe_inspected_frequently with a value of 1 if pipe has an Average_MonthsLastRev of less or equal than 24
train_copy['pipe_inspected_frequently'] = np.where(train_copy['Average_MonthsLastRev'] <= 24, 1, 0)
#place column in 7th position
cols = list(train_copy.columns.values)
cols.pop(cols.index('pipe_inspected_frequently'))
train_copy = train_copy[['PipeId', 'No_Inspections', 'No_Incidents', 'InspectionYear',
         'Probability_rate', 'preventive_maintenance_rate', 'pipe_inspected_frequently', 'InspectionDate', 'Average_MonthsLastRev',
         'MonthsLastRev', 'relative_risk', 'average_severity_pipe', 'Severity',
            'Incidence', 'Province', 'Town', 'YearBuilt', 'Material', 'GasType',
            'Diameter', 'Length', 'Pressure', 'NumConnections',
            'NumConnectionsUnder', 'BoolBridle']]
#delete column Average_MonthsLastRev
train_copy.head(1)


In [ ]:
train_copy.columns

In [ ]:
#creating a column named age_pipe_inspection with the difference between InspectionYear and YearBuilt and placing it in 10 th position
train_copy['Age_pipe_at_inspection'] = train_copy['InspectionYear'] - train_copy['YearBuilt']
cols = list(train_copy.columns.values)
cols.pop(cols.index('Age_pipe_at_inspection'))
train_copy = train_copy[['PipeId', 'No_Inspections', 'No_Incidents', 'InspectionYear',
       'Probability_rate', 'preventive_maintenance_rate', 'Age_pipe_at_inspection',
       'pipe_inspected_frequently', 'InspectionDate', 'Average_MonthsLastRev',
       'MonthsLastRev', 'relative_risk', 'average_severity_pipe', 'Severity',
       'Incidence', 'Province', 'Town', 'YearBuilt', 'Material', 'GasType',
       'Diameter', 'Length', 'Pressure', 'NumConnections',
       'NumConnectionsUnder', 'BoolBridle']]
#head of pipes with Age_pipe_at_inspection greater than 0
train_copy[train_copy['Age_pipe_at_inspection'] > 0].head(2)

In [ ]:
#describe Diameter on train_copy in mm
train_copy['Diameter'].describe()

In [ ]:
train_copy['Length'].describe()

In [ ]:
#divide all values in Diameter by 1000 to convert to meters
train_copy['Diameter'] = train_copy['Diameter'] / 1000

In [ ]:
#create a new variable called aspect that is equal to pressure divided by diameter multiplied by length
train_copy['aspect']=(train_copy['Pressure']/train_copy['Diameter'])/train_copy['Length']

In [ ]:
#create a new column that divides the diameter by the pressure and name it Relative Thickness
train_copy['Relative_Thickness'] = train_copy['Diameter'] / train_copy['Pressure']

In [ ]:
#creating a column named pipe_area that multiplies diameter by lenght by pi

train_copy['pipe_area'] = train_copy['Diameter'] * train_copy['Length'] * 3.1416

In [ ]:
#create a column named area_connection that divides pipe_area by NumConnections
train_copy['area_connection'] = train_copy['NumConnections']/ train_copy['pipe_area'] 

In [ ]:
#create a column named incidence_area that divides incidence by pipe_area
train_copy['incidence_area'] = train_copy['Incidence'] / train_copy['pipe_area']

In [ ]:
#create a boolean column named connection_bool that is 1 if NumConnections is greater than 1
train_copy['connection_bool'] = np.where(train_copy['NumConnections'] > 1, 1, 0)


In [ ]:
#transform Inspection Date to datetime format
train_copy['InspectionDate'] = pd.to_datetime(train_copy['InspectionDate'])


In [ ]:
train_copy.columns

In [ ]:
#hot encode severity_incidence column in the train_copy dataframe
train_copy = pd.get_dummies(train_copy, columns=['Severity'], prefix = ['Severity'])
train_copy.head(1)

In [ ]:
#change name of Severity_Incidence_1 column to Severity_low
train_copy = train_copy.rename(columns={'Severity_1.0': 'Severity_high'})
#change name of Severity_Incidence_2 column to Severity_medium
train_copy = train_copy.rename(columns={'Severity_2.0': 'Severity_medium'})
#change name of Severity_Incidence_3 column to Severity_high
train_copy = train_copy.rename(columns={'Severity_3.0': 'Severity_low'})
#drop Severity_4.0 column
train_copy = train_copy.drop(['Severity_4.0'], axis=1)

In [ ]:
train_copy.head(1)

In [ ]:
train_copy['BoolBridle'].describe()

In [ ]:
# Converting Boolbride into  boolean variable
def boolbridle(x):
    return 1 if x == 'True' else 0

In [ ]:
# Apply function on dataset
train_copy['BoolBridle'] = train_copy['BoolBridle'].apply(lambda x: boolbridle(x))

In [ ]:
#hot encode GasType column in the train_copy dataframe subset
train_copy = pd.get_dummies(train_copy, columns=['GasType'], prefix = ['GasType'])

In [ ]:
#delete GasType_Gas propano column
train_copy = train_copy.drop(['GasType_Gas propano'], axis=1)

In [ ]:
#change name of GasType_Gas natural column to gas_natural
train_copy = train_copy.rename(columns={'GasType_Gas natural': 'gas_natural'})
train_copy.head(1)

In [ ]:
#how much RAM is being used
import psutil
psutil.virtual_memory()


In [ ]:
print (train_copy['Material'].unique())

In [ ]:
#Map df_combined Material column to the names of materials
train_copy['Material'] = train_copy['Material'].map({'PE': 'Polyethylene', 'AO': 'Acrylonitrile-Butadiene-Styrene', 'FD': 'Fiberglass-Reinforced Plastic', 
    'FG': 'Fiberglass', 'PN': 'Polypropylene', 'PA': 'Polyamide', 'FO': 'Flexible Polyolefin', 'FI': 'Flexible Polyvinyl Chloride', 'CU': 'Copper', 
    'PV': 'Polyvinylidene Fluoride', 'ZD': 'Zinc-Coated Steel', 'ZA': 'Zinc-Aluminum', 'CP': 'Cast Iron', 'CS': 'Cast Steel', 
    'ZC': 'Zinc-Coated Steel', 'ZM': 'Zinc-Magnesium','ZN': 'Zinc', 'AL': 'Aluminum', 'ZP': 'Zinc-Coated Steel', 'ZF': 'Zinc-Aluminum-Magnesium'})


In [ ]:
#Hot enconde Material column in the train_copy dataframe subset
train_copy = pd.get_dummies(train_copy, columns=['Material'])
train_copy = train_copy.drop(['Material_Fiberglass', 'Material_Zinc-Coated Steel', 'Material_Polyvinylidene Fluoride','Material_Flexible Polyolefin', 'Material_Flexible Polyvinyl Chloride', 'Material_Polyamide'], axis=1)
train_copy.head(1)

In [ ]:
train_copy.columns

In [ ]:
#create polinomial features in train_copy for Diameter, Length, Pressure
poly = PolynomialFeatures(2)
train_copy['Diameter2'] = poly.fit_transform(train_copy[['Diameter']])[:,2]
train_copy['Length2'] = poly.fit_transform(train_copy[['Length']])[:,2]
train_copy['Pressure2'] = poly.fit_transform(train_copy[['Pressure']])[:,2]
train_copy.head(1)



# Dropping 

Mkaing df lighter just before joining

In [ ]:
train_copy.shape#delete the rows with YearBuilt higher than InspectionYear
train_copy = train_copy.drop(train_copy[(train_copy['YearBuilt'] > train_copy['InspectionYear'])].index)
train_copy.drop('InspectionDate', axis=1, inplace=True)
#show null values in train_copy
train_copy.isnull().sum()
    
#delete nulls from train_copy
train_copy = train_copy.dropna()


In [ ]:
#delete the rows with YearBuilt higher than InspectionYear
train_copy = train_copy.drop(train_copy[(train_copy['YearBuilt'] > train_copy['InspectionYear'])].index)

In [ ]:
#show null values in train_copy
train_copy.isnull().sum()
    

In [ ]:
#delete nulls from train_copy
train_copy = train_copy.dropna()


In [ ]:
# free up memory
del df_inspection
del df_network
del network_unique
del train
del df_sample_submission
del df_sample
del sample_unique

# Adding a new dataset
We want to extract value of two categorical variables, Town and Province but the way we have them now they are useless.

We will add a new dataset to join and extract the surface of each town as well as the comunidad autonoma to group by accordingly in another column the number of towns and afterwards hot encode


In [ ]:
#count towns populating Town column
train_copy['Town'].value_counts()
train_copy['Town'].unique()

In [ ]:
#point to the folder where the data is stored
os.chdir(r"C:\Users\pedro\datathon\base\complementary_datasets")

# Loading provincias dataset
provincias = pd.read_excel('promedio_tiempo_provincia_anual.xlsx')

#Loading municipios dataset
#df_mun = pd.read_excel('list-mun-2012.xls')


In [ ]:
#read population
population = pd.read_excel('pobmun20.xls')


In [ ]:
population.columns

In [ ]:
normalize_characters(population, 'Province')

In [ ]:
normalize_characters(population, 'Municipio')

In [ ]:
#change Municipio column with Town column
population = population.rename(columns={'Municipio': 'Town'})


In [ ]:
provincias.head(1)

In [ ]:
normalize_characters(provincias, 'Province')

In [ ]:
normalize_characters(train_copy, 'Province')

In [ ]:
#join train_copy and provincias on Province
train_copy = train_copy.merge(provincias, on='Province', how='left')

In [ ]:
train_copy.head(1)

In [ ]:
#count nulls in train_copy
train_copy.isnull().sum()

In [ ]:
#compare array for column Province in train_copy and provincias and compute the set difference
np.setdiff1d(train_copy['Province'].unique(), provincias['Province'].unique())


In [ ]:
#place Province and Town Columns at the end of the dataframe
cols = list(train_copy.columns.values)
cols.pop(cols.index('Province'))
cols.pop(cols.index('Town'))
train_copy = train_copy[cols+['Province','Town']]
train_copy.tail(2)

In [ ]:
#join train_copy and population on Town considering the best match
train_copy = train_copy.merge(population, on='Town', how='left')

#count nulls in train_copy
train_copy.isnull().sum()

In [ ]:
train_copy.columns

In [ ]:
#pip install fuzzyjoin
#pip install fuzzymatcher
#pip install fuzzy_pandas

In [ ]:
from fuzzymatcher import link_table, fuzzy_left_join
import fuzzyjoin
import difflib
import fuzzy_pandas as fpd

In [ ]:
#use fuzzypandas to merge train_copy and population on Town 
train_copy = fpd.fuzzy_merge(train_copy, population, left_on='Town', right_on='Town', method='levenshtein', threshold=0.6, keep='match', join="left-outer")

In [ ]:
#droping df_mun from memory to free RAM
del df_mun
del train
del df_inspection
del df_network


In [ ]:
train_copy.dtypes

# Outlier section

In [ ]:
#delete outliers in Year_Built column < 1960
train_copy = train_copy[train_copy['YearBuilt'] > 1960]

In [ ]:
#Show the distribution of YearBuilt
plt.figure(figsize=(10,5))
sns.distplot(train_copy['YearBuilt'], color='blue', bins=100, hist_kws={'alpha': 0.4});
plt.show()

In [ ]:
#eliminate values higher than 40 in MonthsLastRev column
train_copy = train_copy[train_copy['MonthsLastRev'] < 40]

In [ ]:
#show the distribution of MonthsLastRev
plt.figure(figsize=(10,5))
sns.distplot(train_copy['MonthsLastRev'], color='blue', bins=100, hist_kws={'alpha': 0.4});
plt.show()


In [ ]:
# use describe function of monthslastrev column to see the distribution of values
train_copy['MonthsLastRev'].describe()


In [ ]:
#eliminate values higher than 400 in diameter column
train_copy = train_copy[train_copy['Diameter'] < 400]

In [ ]:
#show the distribution of Diameter
plt.figure(figsize=(10,5))
sns.distplot(train_copy['Diameter'], color='blue', bins=100, hist_kws={'alpha': 0.4});
plt.show()


In [ ]:
cols = ['Age_pipe_at_inspection','Length','YearBuilt','InspectionYear'] # one or more

Q1 = train_copy[cols].quantile(0.25)
Q3 = train_copy[cols].quantile(0.75)
IQR = Q3 - Q1

train_copy = train_copy[~((train_copy[cols] < (Q1 - 1.5 * IQR)) |(train_copy[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
train_copy.shape

In [ ]:
#graph histogram of MonthsLastRev column
train_copy['MonthsLastRev'].hist()

In [ ]:
#plot column Length
train_copy['Length'].value_counts()

In [ ]:
#plot histogram of Length column
train_copy['Length'].hist()

In [ ]:
train_copy.columns

# Dropping section
Dropping will be done here

In [ ]:
#delete the rows with YearBuilt higher than InspectionYear
train_copy = train_copy.drop(train_copy[(train_copy['YearBuilt'] > train_copy['InspectionYear'])].index)

In [ ]:
train_copy.drop('InspectionDate', axis=1, inplace=True)

In [ ]:
#show null values in train_copy
train_copy.isnull().sum()
    

In [ ]:
#delete nulls from train_copy
train_copy = train_copy.dropna()


# Performing dimensionality reduction

In [ ]:
# do a pca to reduce the number of columns in train_copy
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(train_copy)
pca.explained_variance_ratio_

#show the columns of train_copy
train_copy.columns




# Exporting the new dataset for a next notebook

In [ ]:
#point to the folder where the data is stored
os.chdir(r"C:\Users\pedro\datathon")
#export the dataframe to a csv file
train_copy.to_csv('train_consolidated.csv', index=False)

# Plotting EDA for new dataset

In [ ]:
#pandas profiling on the train_copy dataframe
profile = ProfileReport(train_copy, title='Pandas Profiling Report', html={'style':{'full_width':True}})
profile

# The Archive Section
Just in case we regret....
NOT DELETED JUST IN CASE NEEDED, HIGHLY CONFIDENT WE DONT NEED THESE


#aggregate the number of Age_pipe_inspection in a new column
train_copy['Aggregate_pipe_age_inspection'] = train_copy.groupby('PipeId')['Age_pipe_inspection'].transform('sum')
#place column in 10th position
cols = list(train_copy.columns.values)
cols.pop(cols.index('Aggregate_pipe_age_inspection'))
train_copy = train_copy[['PipeId', 'MaintenanceId', 'Inspections', 'Probability_incidence', 'Risk_S*I/Inspections', 'Average_MonthsLastRev',
    'Age_pipe_inspection', 'Aggregate_pipe_age_inspection', 'No_Incidents', 'Risk_S*I', 'average_severity', 'MonthsLastRev', 'Severity', 'Incidence', 'Province',
    'Town','YearBuilt', 'InspectionYear', 'InspectionDate','Material', 'GasType', 'Diameter', 'Length', 'Pressure',
    'NumConnections', 'NumConnectionsUnder', 'BoolBridle']]
train_copy.head(10)


#divide the Aggregate_pipe_age_inspection by the count of Incidence when is 1 and create new column with the average_age_pipe_inspection_when_incidence saving it in 10th position
train_copy['average_age_pipe_inspection_when_incidence'] = train_copy['Aggregate_pipe_age_inspection'] / train_copy.groupby('PipeId')['Incidence'].transform('count')
#place column in 10th position
cols = list(train_copy.columns.values)
cols.pop(cols.index('average_age_pipe_inspection_when_incidence'))
train_copy = train_copy[['PipeId', 'MaintenanceId', 'Inspections', 'Probability_incidence', 'Risk_S*I/Inspections', 'Average_MonthsLastRev',
    'Age_pipe_inspection', 'Aggregate_pipe_age_inspection', 'average_age_pipe_inspection_when_incidence', 'No_Incidents', 'Risk_S*I', 'average_severity', 'MonthsLastRev', 'Severity', 'Incidence', 'Province',
    'Town','YearBuilt', 'InspectionYear', 'InspectionDate','Material', 'GasType', 'Diameter', 'Length', 'Pressure',
    'NumConnections', 'NumConnectionsUnder', 'BoolBridle']]
#head of rows only where average_age_pipe_inspection_when_incidence when Incidence is 0
train_copy[train_copy['Incidence'] == 0].head(10)

#take PipeId column from df_submission dataframe and match it with PipeId column in train_copy dataframe to split train_copy into df_combined_train and df_combined_test. The test split should be the one with the higher number of rows
df_combined_test = train_copy[train_copy['PipeId'].isin(df_submission['PipeId'])]
df_combined_train = train_copy[~train_copy['PipeId'].isin(df_submission['PipeId'])]
#Do not look back at the test set until you are ready to submit your predictions


#take PipeId column from df_submission dataframe and match it with PipeId column in train_copy dataframe to split train_copy into df_combined_train and df_combined_test. The test split should be the one with the higher number of rows
df_combined_trained = train_copy[train_copy['PipeId'].isin(df_submission['PipeId'])]
df_combined_test = train_copy[~train_copy['PipeId'].isin(df_submission['PipeId'])]

#normalize df_combined_train_sub dataframe but the booleans
df_combined_train_sub_norm = df_combined_train_sub.copy()
df_combined_train_sub_norm[['Inspections', 'No_Incidents', 'Risk_S*I/Inspections', 'MonthsLastRev', 'Risk_S*I', 'Severity',
         'YearBuilt', 'Diameter', 'Length', 'Pressure', 'NumConnections','NumConnectionsUnder', 'TownCount']] = MinMaxScaler().fit_transform(df_combined_train_sub_norm[['Inspections', 'No_Incidents', 'Risk_S*I/Inspections', 'MonthsLastRev', 'Risk_S*I', 'Severity',
            'YearBuilt', 'Diameter', 'Length', 'Pressure', 'NumConnections','NumConnectionsUnder', 'TownCount']])
df_combined_train_sub_norm.head(2)

#Extract the day of the week from InspectionDate and place it in 5th position
train_copy['InspectionDay'] = train_copy['InspectionDate'].dt.day_name()
cols = list(train_copy.columns.values)
cols.pop(cols.index('InspectionDay'))
train_copy = train_copy[['PipeId', 'Inspections', 'No_Incidents', 'Risk_S*I/Inspections','leakage_estimate_factor','InspectionDay',
    'InspectionYear', 'InspectionDate', 'MonthsLastRev', 'Risk_S*I','Severity','Incidence', 'Province', 'Town', 'YearBuilt', 'Material', 'GasType',
    'Diameter', 'Length', 'Pressure', 'NumConnections', 'NumConnectionsUnder', 'BoolBridle']]
train_copy.head(1)


#on the train_copy if YEarBuilt is equal to 2021 and InspectionYear is equal to NaN, fill the InspectionYear with 2021
#train_copy.loc[(train_copy['YearBuilt'] > 2020) & (train_copy['InspectionYear'].isnull()), 'InspectionYear'] = 2021


#do a range in the train_copy for the YearBuilt column if the value is greater than 2020 and the InspectionYear is equal to Nan, change the value of InspectionYear to YearBuilt

for i in train_copy['YearBuilt']:
    if i > 2020:
        train_copy.loc[(train_copy['YearBuilt'] > 2020) & (train_copy['InspectionYear'].isnull()), 'InspectionYear'] = i